In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

%matplotlib inline

In [3]:
from YahooFinanceHelper import YahooHistoricalData

## Grab SP500 ticker list from wikipedia

In [4]:
urlString = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
url = urlopen(urlString)
soup = BeautifulSoup(url, 'html.parser')
print(soup.title)

<title>List of S&amp;P 500 companies - Wikipedia</title>


In [5]:
first_table = soup.select_one('table:nth-of-type(1)')
tableBody = first_table.find('tbody')
tableRows = tableBody.find_all('tr')

In [6]:
SP500 = []

for i in range(1, len(tableRows)):
    temp = tableRows[i].find('a').text.strip()
    SP500.append(temp)

## Run through list to gather adjusted close from every SP500 stock

In [8]:
df = pd.DataFrame()

for i in SP500:
    data = YahooHistoricalData(i.replace('.', '-'), days_back=365).get_quote()
    #print(i)
    #if stocks.index(i) == 0:
    #    df['Date'] = data['Date']
    df[i] = data['Adj Close']

ReadTimeout: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Read timed out. (read timeout=2)

## Upload data to Heroku Postgresql, append if the row exists

In [ ]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [ ]:
dotenv_path = 'env_vars2.env'
load_dotenv(dotenv_path)

In [ ]:
URI = os.getenv('URI')

In [ ]:
engine = create_engine(URI)

In [ ]:
df.to_sql('close', engine, if_exists='append')